In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess
from tensorflow.keras.applications import densenet
import numpy as np
import os, gc

app = Flask(__name__)
CORS(app)

# -----------------------------------------
# LOAD MODELS
# -----------------------------------------
ResNet_Path = './chest_CT_SCAN-ResNet50.keras'
DenseNet_Path = './chest_CT_SCAN-DenseNet201.keras'

print("🔹 Loading models...")
ResNet_model = load_model(ResNet_Path)
DenseNet_model = load_model(DenseNet_Path)
print("✅ Models loaded")

# Class labels
classes = ["Adenocarcinoma", "Large cell carcinoma", "Normal", "Squamous cell carcinoma"]

UPLOAD_DIR = "uploads"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# -----------------------------------------
# PREDICTION HELPER
# -----------------------------------------
def predict_class(img_array):
    # --- ResNet ---
    resnet_input = resnet_preprocess(img_array.copy())
    resnet_pred = ResNet_model.predict(resnet_input)
    resnet_conf = float(np.max(resnet_pred))
    resnet_idx = int(np.argmax(resnet_pred))

    # --- DenseNet ---
    dense_input = densenet_preprocess(img_array.copy())
    dense_pred = DenseNet_model.predict(dense_input)
    dense_conf = float(np.max(dense_pred))
    dense_idx = int(np.argmax(dense_pred))

    # pick stronger prediction
    if resnet_conf >= dense_conf:
        return resnet_idx, "ResNet50", resnet_conf
    else:
        return dense_idx, "DenseNet201", dense_conf


# -----------------------------------------
# API ROUTE
# -----------------------------------------
@app.route("/analyze", methods=["POST"])
def analyze():
    try:
        if "image" not in request.files:
            return jsonify({"error": "No image uploaded"}), 400

        file = request.files["image"]
        filepath = os.path.join(UPLOAD_DIR, file.filename)
        file.save(filepath)

        # Load image
        img = image.load_img(filepath, target_size=(460, 460))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)

        # Predict
        pred_idx, model_used, confidence = predict_class(img_array)
        label = classes[pred_idx]

        gc.collect()

        return jsonify({
            "classification": label.lower(),
            "confidence": 1
        })

    except Exception as e:
        gc.collect()
        return jsonify({"error": str(e)}), 500


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5001, debug=False, use_reloader=False)


ModuleNotFoundError: No module named 'flask'